In [1]:
import os
import copy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scprep
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from sklearn.neighbors import kneighbors_graph
from scipy.sparse.csgraph import shortest_path, connected_components

In [2]:
import phate
from sklearn.manifold import TSNE
import networkx as nx
from matplotlib.patches import Patch

In [3]:
import helpers

In [4]:
merged_metadata, relatedness, genotypes_array, mapping_info = helpers.load_data()

# Step 0: Pre-process data
normalized_matrix, overlap_counts = helpers.preprocess_data_matrix(genotypes_array)

# Fit PCA model on unrelated samples
filters = ["filter_pca_outlier", "hard_filtered", "filter_contaminated"]
_filtered_indices = merged_metadata[merged_metadata[filters].any(axis=1)].index
filtered_indices = ~merged_metadata.index.isin(_filtered_indices)
related_indices = ~merged_metadata['filter_king_related'].values #np.ones(shape=genotypes_array.shape[0], dtype=bool)

to_fit_on = related_indices & filtered_indices
to_transform_on = (~related_indices) & filtered_indices

pca_emb, _ = helpers.compute_pca_from_hail('/lustre06/project/6065672/shared/MattDataSharing/1KGP+HGDP/V4/pca_scores_hailcomputed.csv', 
                                            merged_metadata, 
                                            50)

Loading previously computed non-missing overlap matrix...


In [5]:
def compute_phate(pca_input, to_fit_on, to_transform_on, **phate_params):
    phate_emb = np.zeros((len(pca_input), 2))

    # Step 4: Run PHATE on PCA-reduced data
    phate_operator = phate.PHATE(random_state=42, n_pca=None, **phate_params)
    
    # Fit PHATE on either the filtered unrelated or all data, based on phate_fit_related
    phate_operator.fit(pca_input[to_fit_on, :])
    
    # Transform all filtered individuals using PHATE embedding
    phate_emb[to_fit_on] = phate_operator.transform(pca_input[to_fit_on, :])
    phate_emb[to_transform_on] = phate_operator.transform(pca_input[to_transform_on, :])
    
    return phate_emb, phate_operator

In [6]:
import sk_sugar

In [7]:
# Github: https://github.com/KrishnaswamyLab/sk_sugar/blob/master/sk_sugar/core.py
# not clear what I do with this!
dir(sk_sugar)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__']

In [8]:
os.chdir('/lustre06/project/6065672/sciclun4/Local/SUGAR/python')
import sugar

In [9]:
sugar_obj = sugar.SUGAR()

In [10]:
new_data = sugar_obj.fit(pca_emb)

Calculating generation estimate...
  Calculating sparsity estimate...
    Calculating sparsity kernel...
      Calculating distance matrix...
      Calculated distance matrix in 0.30 seconds.
    Calculated sparsity kernel in 0.77 seconds.
  Calculated sparsity estimate in 0.78 seconds.
Calculated generation estimate in 0.78 seconds.


In [11]:
new_data = sugar_obj.transform(pca_emb)

Calculating covariance estimation...
Calculated covariance estimation in 0.34 seconds.


LinAlgError: SVD did not converge

In [12]:
sugar_obj._gen_est.max(), sugar_obj._gen_est.min()

(0, 0)

In [13]:
sugar_obj.generate_points(X = sugar_obj.X, gen_est = sugar_obj._gen_est)

LinAlgError: SVD did not converge

In [ ]:
# Code in question
# from: https://github.com/stanleyjs/SUGAR/blob/develop/python/sugar.py
# if X is None:
#     X = self.X
# if gen_est is None:
#     gen_est = self.generation_estimate
# Y_random = np.ndarray((np.sum(gen_est), X.shape[1]))
# cur_idx = 0
# for ix, ell in enumerate(gen_est):
#     Y_random[cur_idx:cur_idx + ell, :] = \
#         np.random.multivariate_normal(self.X[ix, :],
#                                       self.covs[ix],
#                                       ell)